# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

##### ✅ Answer:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun


tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
    WikipediaQueryRun(top_k_results=3),
]

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

/var/folders/zf/2_xs6smx5417c983wzwm2k1r0000gn/T/ipykernel_35022/1203815797.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [ ]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

##### ✅ Answer:
This function is called for each item in the tool_belt list. It converts these items into a format that is compatible with OpenAI's models. Binding, in this context, means that the model is now able to call these tool and decide which one to use

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [8]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [9]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [10]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [11]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [12]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [13]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [14]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

##### ✅ Answer:
Graph edges
Conditional logic between nodes
focus : Where the decision happens → on the edges between nodes

No,  we can have a condition edge says that for example go through cycle 2 times only then exit(it is while loop in context)
Storing a counter in the state object


Keep a Counter in State
focus : How  keep track of loops → using a counter in state.
The simplest way:
Store a counter in AgentState.
Increment it on each loop.
Break out when the limit is reached.


Use a Graph Runner Timeout or Max Iterations
Some orchestration tools (LangGraph included) support:
timeout → max seconds to run
max iterations → max steps or transitions allowed

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [15]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vTrao1341QmoXcrpm0bkOK1I', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 162, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BssoKLpDh5mCMEFCFRBPnNvoPPPfn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9dd9f660-1c06-4809-9297-8c40ee1a3467-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_vTrao1341QmoXcrpm0bkOK1I', 'type': 

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [16]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_I09qRNR1EcRftBPlOoEorl1d', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_SebiQqPjdaBD1lkz2xX7UJl3', 'function': {'arguments': '{"query": "latest Tweet of the first author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_aU7tdGOAnbYI02opJI8j1Nwj', 'function': {'arguments': '{"query": "latest Tweet of the second author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_wIAeHnonkGz5dg2qoNGqWnhC', 'function': {'arguments': '{"query": "latest Tweet of the third author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 178, 'total_tokens': 291, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0,

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

##### ✅ Answer:
Step 1 — User’s Question Arrives
A user sends the question :

“Find details about QLoRA and the latest tweets from its authors.”

This becomes a HumanMessage stored in the state.

Step 2 — Agent Receives the State
The agent node reads the latest message in the state.

It reasons:

“I need to look up information about QLoRA.”

“I also need to find the latest tweets from the authors.”

Step 3 — Agent Decides on Tool Calls
Instead of directly answering, the agent:
✅ decides to call tools.

This is evident in log:

json
Copy
Edit
'tool_calls': [
  {
    'name': 'arxiv',
    'args': {'query': 'QLoRA'},
    ...
  },
  {
    'name': 'tavily_search_results_json',
    'args': {'query': 'latest Tweet of the first author of QLoRA'},
    ...
  },
  {
    'name': 'tavily_search_results_json',
    'args': {'query': 'latest Tweet of the second author of QLoRA'},
    ...
  },
  {
    'name': 'tavily_search_results_json',
    'args': {'query': 'latest Tweet of the third author of QLoRA'},
    ...
  }
]
So the agent:

generates Tool Calls for:

arxiv search

Tavily search for tweets

These tool calls get appended as AgentMessage into the state.

Step 4 — Graph Runner Executes Tools
Now the graph runner:
✅ reads the tool calls.
✅ dispatches each tool call:

sends the query to the relevant tool (e.g. Arxiv, Tavily)

Step 5 — Tools Return Results
Each tool returns its result as a ToolMessage.

Example:

✅ Arxiv returns the QLoRA paper summary.

✅ Tavily returns tweets and articles about QLoRA authors.

These tool messages are appended to the state:

scss
Copy
Edit
ToolMessage(content='Published: 2023-05-23 ... QLoRA...')
ToolMessage(content='[{"title": "..."}]')
...
Step 6 — Agent Reads Tool Outputs
After tools finish:
✅ the agent node runs again.

It reads the updated state:

sees the ToolMessages.

extracts:

paper info

tweet data

Step 7 — Agent Generates the Final Answer
Finally, the agent composes a complete answer.


This answer becomes an AIMessage stored in the state.

✅ The graph runner then completes the flow.

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [17]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

In [18]:
agent_chain_with_formatting.invoke({"question" : "What is RAG?"})

"RAG can refer to different concepts depending on the context. Could you please specify whether you're asking about RAG in the context of project management, machine learning, or another field?"

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [19]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]

##### ✅ Answer:

In [21]:
questions = [
    "Can QLORA match full 16-bit finetuning performance at 33B and 65B scales.?",
    "What is more important for training chatbot models, according to the study on QLORA?",
    "What techniques does QLORA use to achieve high-fidelity 4-bit finetuning?",
    "How does 4-bit QLORA's performance compare to 16-bit methods according to the study?",
    "What breakthrough does QLORA achieve in finetuning large AI models?"
]

answers = [
    {"must_mention" : ["resource", "limitation"]},
    {"must_mention" : ["Size", "quality"]},
    {"must_mention" : ["4-bit NormalFloat", "Double Quantization,"]},
    {"must_mention" : ["finetuning", "benchmark"]},
    {"must_mention" : ["GPUs", "NLP"]},
]


Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [22]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['cb191bd4-905a-451b-be0e-1fde72d7b596',
  '43b99a27-9fd7-4f2a-962b-1addbbeaca9c',
  '9c40c90c-9fba-438e-a8b7-9cfb8ef3b73c',
  '7707872c-24f0-4a15-b51d-5a5a0123c8c6',
  '374d12ce-7c3f-4df3-93ce-1304794479fd'],
 'count': 5}

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

##### ✅ Answer:
with create a dataset contains quesions and answers and the answers must contains specific phrase. so out of 5 of them only 3rd one receive full score the others were only incomplete , mentioned one of them

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [25]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

##### ✅ Answer:

There are 2 broad ways 

***use : code-based keyword checks***
1-Lowercase and Strip Whitespace
2-Use Synonyms or Keyword Variants
3-Allow Partial Matches(from rapidfuzz import fuzz
def is_close_enough(a, b, threshold=80):
    return fuzz.partial_ratio(a.lower(), b.lower()) >= threshold)

4-semantic similiarity 
check meaning rather than exact words.
Example:
Embed required terms
Embed the model output
Compute cosine similarity
If similarity > threshold → consider it a match.

***use LLM as Judge***
criteria → LLM scoring answer quality.
cot_qa → LLM reasoning to check correctness.(Chain-of-Thought QA)

from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
    evaluators=[
        "criteria",
        "cot_qa",
    ],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [26]:
experiment_results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"Search Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'Search Pipeline - Evaluation - a62d-59233d33' at:
https://smith.langchain.com/o/fbdffdce-5f8b-5660-ad4e-686bfc2dced2/datasets/b23396eb-ce87-4436-a816-43dd17f4455e/compare?selectedSessions=cbd86b03-e47f-40f5-a9f0-b68909267dd3




0it [00:00, ?it/s]

In [28]:
experiment_results

<ExperimentResults Search Pipeline - Evaluation - a62d-59233d33>

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [29]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE

In [31]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

***✅ Answer***


**graph_with_helpfulness_check = StateGraph(AgentState)**  
    - Creates a new graph that tracks state between nodes using the `AgentState` TypedDict.  
    - The `AgentState` object will hold conversation messages, intermediate variables, and evaluation results.

- **add_node("agent", call_model)**  
    - Adds a node named `"agent"`.  
    - This node will execute the `call_model` function, which typically calls a language model to generate a response based on the current conversation state.

- **add_node("action", tool_node)**  
    - Adds a node named `"action"`.  
    - This node will execute the `tool_node` function, which could call external tools or APIs (e.g. search engines, calculators, retrievers).

Together, these nodes define the **steps** in the graph workflow. 


##### YOUR MARKDOWN HERE

In [34]:
graph_with_helpfulness_check.set_entry_point("agent")

***✅Answer***
 It tells the graph:
Begin execution at the node named `"agent"`.

##### YOUR MARKDOWN HERE

In [35]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

***✅Answer***
- Checks if the LLM output includes any tool calls:
    - If yes → the graph transitions to the `"action"` node to run external tools.

- Checks if the conversation has become too long (over 10 messages):
    - If yes → the graph ends to avoid infinite loops.

- Otherwise, evaluates whether the model’s last response is helpful:
    - Builds a simple prompt asking the LLM:
      > "Given the initial user query and the final response, is the response extremely helpful? Answer Y or N."
    - Calls the LLM to get the answer ("Y" or "N").
    - If the answer contains "Y":
        - The graph ends.
    - Otherwise:
        - The graph loops back to continue generating a better answer.

#### 🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

***✅Answer***

This statement defines **dynamic routing** in our LangGraph workflow:

- After running the `"agent"` node:
    - We call the function `tool_call_or_helpful(state)`.
    - Depending on what it returns:
        - `"continue"` → loop back to `"agent"` for another answer attempt.
        - `"action"` → transition to the `"action"` node to run external tools.
        - `"end"` → terminate the graph.

**Inside `tool_call_or_helpful`:**
- Checks if the last message includes tool calls → routes to `"action"`.
- Checks if the conversation is too long → routes to `"end"`.
- Otherwise, performs an LLM-based helpfulness check:
    - If helpful → routes to `"end"`.
    - Otherwise → loops back to `"agent"`.

##### YOUR MARKDOWN HERE

In [36]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

In [37]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

***✅Answer***

After finishing the node "action", go to the node "agent".

In [39]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

***✅Answer***

Run Graph Asynchronously and Stream Updates

- Sends an initial user message into the LangGraph.
- Streams updates after each node runs.
- Prints which node produced output and the current messages in the conversation.
- Useful for debugging and tracing graph execution step by step


This allows us to **trace the entire conversation flow** through:
- the agent’s responses
- any tool calls
- loops in the graph
- evaluation decisions

It’s extremely helpful for debugging and understanding how iur graph behaves step by step.

### Why Use Streaming?

✅ **Streaming updates** let us:
- See intermediate steps from each node
- Debug conditional routing logic
- Monitor which tools are being called
- Observe loops and state changes in real time

In [40]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rbMIx1olqHrrORraOQtUmxOh', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_nikcEJcueak99AXe9GJhLC2H', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_OgEpuEx7v2KEEga7tYLhcQDd', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 177, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-Bsx

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [41]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [42]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is the process of designing and refining input prompts to effectively communicate with AI language models, such as GPT, to elicit accurate, relevant, and useful responses. It involves crafting prompts that guide the model's output in a desired direction, often through techniques like specifying context, framing questions clearly, or using specific keywords.

Prompt engineering has gained prominence with the rise of large language models (LLMs) around 2020 and 2021, as these models became more capable and widely adopted. The practice became especially important as users and developers sought to maximize the utility of these models across various applications, from chatbots to content generation.

Would you like me to find more detailed historical information or recent developments related to prompt engineering?



RAG, which stands for Retrieval-Augmented Generation, is a technique in natural language processing that combines pre-trained language models with informati